In [1]:
%matplotlib widget

import os
import sys

sys.path.insert(0, os.getenv("HOME") + "/pycode/MscThesis/")
import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_plate_number,
    get_postion_number,
    get_begin_index,
)
import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    sparse_to_doc,
)
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import (
    Experiment,
    Edge,
    Node,
    Hyphae,
    plot_raw_plus,
    get_hyphae,
)
from amftrack.pipeline.paths.directory import (
    run_parallel,
    find_state,
    directory_scratch,
    directory_project,
)
from amftrack.notebooks.analysis.util import *
from scipy import stats
from scipy.ndimage.filters import uniform_filter1d
from statsmodels.stats import weightstats as stests
from amftrack.pipeline.functions.hyphae_id_surf import (
    get_pixel_growth_and_new_children,
    resolve_ambiguity,
    relabel_nodes_after_amb,
)
from collections import Counter
from IPython.display import clear_output
from amftrack.notebooks.analysis.data_info import *
from amftrack.pipeline.functions.node_id import reconnect_degree_2
from amftrack.pipeline.functions.extract_graph import (
    generate_skeleton,
    from_nx_to_tab,
    prune_graph,
)
from amftrack.pipeline.functions.hyphae_id_surf import (
    clean_and_relabel,
    get_mother,
    save_hyphaes,
    resolve_ambiguity_two_ends,
    clean_obvious_fake_tips,
    width_based_cleaning,
)
from amftrack.pipeline.functions.experiment_class_surf import clean_exp_with_hyphaes

In [1]:
directory = directory_project
plate_number = 102
plate = get_postion_number(plate_number)
begin = 262
end = 287
dates_datetime = get_dates_datetime(directory, plate)
dates = dates_datetime[begin : end + 1]
exp = Experiment(plate, directory)
exp.load(dates)  # for method 2

NameError: name 'directory_project' is not defined

In [3]:
exp.load_compressed_skel()

In [4]:
experiment = exp
exclude_bottom_factor = 0.98
tips = [
    node
    for node in experiment.nodes
    if node.degree(node.ts()[0]) == 1
    and node.pos(node.ts()[0])[0] <= experiment.boundaries_x[1] * exclude_bottom_factor
]
problems2 = []
problems = []

hyphaes = []
select = {}
#     for i in range(20):
for i, tip in enumerate(tips):
    if i % 200 == 0:
        print(i / len(tips))
    #         tip = choice(tips)
    hyphae = Hyphae(tip)
    roots = []
    for t in tip.ts():
        #             print(t,tip)
        if tip.degree(t) == 1:
            root, edges, nodes = hyphae.get_edges(t, 200)
            roots.append(root)
    occurence_count = Counter(roots)
    if len(occurence_count.values()) >= 2:
        select[tip] = occurence_count
    if (
        len(occurence_count.values()) >= 2
        and occurence_count.most_common(2)[0][0] != roots[0]
        and occurence_count.most_common(2)[1][1] / occurence_count.most_common(2)[0][1]
        >= 0.75
    ):
        problems2.append(tip)
    else:
        hyphae.root = occurence_count.most_common(2)[0][0]
        if hyphae.root != roots[0]:
            problems.append((tip, roots[0]))
        hyphae.ts = sorted(set(hyphae.ts).intersection(set(hyphae.root.ts())))
        hyphaes.append(hyphae)

0.0
0.14104372355430184
0.2820874471086037
0.4231311706629055
0.5641748942172073
0.7052186177715092
0.846262341325811
0.9873060648801129


In [38]:
len(problems)

131

In [43]:
tip, root = choice(problems)

In [13]:
len(tips), len(select.keys())

(1418, 384)

In [35]:
tips = select.keys()
tip = choice(list(tips))

In [44]:
tip, select[tip], tip.ts()

(Node(2184), Counter({Node(2183): 1, Node(1884): 2}), [13, 14, 15])

In [45]:
plt.close("all")
plot_raw_plus(
    exp, tip.ts()[0], [tip.label] + [node.label for node in select[tip].keys()]
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
exp.plot([tip.ts()[0] + 2], [[tip.label] + [node.label for node in select[tip].keys()]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IndexError: list index out of range

In [64]:
experiment = exp
ts = {}
nx_graph_cleans = [nx.Graph.copy(nx_g) for nx_g in experiment.nx_graph]
exp_clean = experiment
labels = {node for g in exp_clean.nx_graph for node in g}
exp_clean.nodes = []
for label in labels:
    exp_clean.nodes.append(Node(label, exp_clean))
for node in exp_clean.nodes:
    ts[node.label] = []
    for t in range(len(nx_graph_cleans)):
        if node.is_in(t):
            ts[node.label].append(t)
hyphaes, problems = get_hyphae(exp_clean)

0.0
0.02884338044418806
0.05768676088837612
0.08653014133256418
0.11537352177675224
0.1442169022209403
0.17306028266512835
0.2019036631093164
0.23074704355350448
0.25959042399769255
0.2884338044418806
0.31727718488606865
0.3461205653302567
0.37496394577444475
0.4038073262186328
0.4326507066628209
0.46149408710700895
0.490337467551197
0.5191808479953851
0.5480242284395731
0.5768676088837612
0.6057109893279492
0.6345543697721373
0.6633977502163253
0.6922411306605134
0.7210845111047015
0.7499278915488895
0.7787712719930776
0.8076146524372656
0.8364580328814537
0.8653014133256418
0.8941447937698298
0.9229881742140179
0.9518315546582059
0.980674935102394
Detected problems during hyphae detection, 80 hyphaes have inconsistent root over time


In [27]:
exp_clean.load_compressed_skel()

In [65]:
to_remove = []
safe_nodes = set()
roots = set()
tips = set()
to_remove_hyphae = set()
for hyphae in hyphaes:
    roots.add(hyphae.root.label)
    tips.add(hyphae.end.label)
    for t in hyphae.end.ts():
        if t in hyphae.root.ts():
            try:
                for node in hyphae.get_nodes_within(t)[0]:
                    safe_nodes.add(node)
            except nx.exception.NetworkXNoPath:
                to_remove_hyphae.add(hyphae)
                print(
                    f"error for hyphae {hyphae} on position {hyphae.end.pos(t),hyphae.root.pos(t)}"
                )
for hyphae in to_remove_hyphae:
    hyphaes.remove(hyphae)
for node in experiment.nodes:
    posit = node.pos(ts[node.label][0])
    if (
        len(ts[node.label]) == 1
        and ts[node.label][0] != len(exp_clean.nx_graph) - 1
        and node.label not in safe_nodes
    ):
        to_remove.append(node.label)
#     return(to_remove)
print(
    "removing ",
    len(to_remove),
    f" nodes out of {len(exp_clean.nodes)} because they appear only in one timestep and are not within an identified hypha",
)
# for node in to_remove:
#     t = ts[node][0]
#     pos = exp_clean.positions[t]
#     nx_graph_clean = nx_graph_cleans[t]
#     if nx_graph_clean.degree(node) <= 2:
#         continue
#     #         print('cleaning',node)
#     neighbours = list(nx_graph_clean.neighbors(node))
#     candidate_to_fuse = []
#     weight_candidate = []
#     for neighbour in neighbours:
#         if nx_graph_clean.degree(neighbour) >= 3:
#             candidate_to_fuse.append(neighbour)
#             weight_candidate.append(
#                 len(nx_graph_clean.get_edge_data(node, neighbour)["pixel_list"])
#                 - 100 * (neighbour in roots)
#                 - 100 * (neighbour in tips)
#             )
#     if len(weight_candidate) == 0:
#         continue
#     node_to_fuse = candidate_to_fuse[np.argmin(weight_candidate)]
#     if nx_graph_clean.degree(node) == 1 and node_to_fuse not in to_remove:
#         print(pos[node])
#         continue
#     for neighbour in neighbours:
#         right_n = node_to_fuse
#         left_n = neighbour
#         right_edge = nx_graph_clean.get_edge_data(node, right_n)["pixel_list"]
#         left_edge = nx_graph_clean.get_edge_data(node, left_n)["pixel_list"]
#         right_edge_width = nx_graph_clean.get_edge_data(node, right_n)["width"]
#         left_edge_width = nx_graph_clean.get_edge_data(node, left_n)["width"]
#         if np.any(right_edge[0] != pos[node]):
#             right_edge = list(reversed(right_edge))
#         if np.any(left_edge[-1] != pos[node]):
#             left_edge = list(reversed(left_edge))
#         pixel_list = left_edge + right_edge[1:]
#         width_new = (right_edge_width*len(right_edge)+left_edge_width*len(left_edge))/(len(right_edge)+len(left_edge))
# #             print(width_new)
#         info = {"weight": len(pixel_list), "pixel_list": pixel_list, "width" : width_new}
#         if right_n != left_n:
#             connection_data = nx_graph_clean.get_edge_data(right_n, left_n)
#             if (
#                 connection_data is None
#                 or connection_data["weight"] >= info["weight"]
#             ):
#                 if not connection_data is None:
#                     nx_graph_clean.remove_edge(right_n, left_n)
#                 nx_graph_clean.add_edges_from([(right_n, left_n, info)])
#     nx_graph_clean.remove_node(node)
#     nx_graph_cleans[t] = nx_graph_clean
for t, nx_graph in enumerate(nx_graph_cleans):
    pos = exp_clean.positions[t]
    reconnect_degree_2(nx_graph, pos)
nx_graph_pruned = []
for graph in nx_graph_cleans:
    nx_graph_pruned.append(prune_graph(graph, 0.1))
# skeletons = []
# for nx_graph in nx_graph_pruned:
#     skeletons.append(generate_skeleton(nx_graph, dim=(30000, 60000)))
exp_clean.nx_graph = nx_graph_pruned
# exp_clean.skeletons = skeletons
labels = {node for g in exp_clean.nx_graph for node in g}
exp_clean.nodes = []
for hyphae in hyphaes:
    hyphae.update_ts()
to_remove_hyphae = set()
for hyphae in hyphaes:
    for t in hyphae.end.ts():
        if t in hyphae.root.ts():
            try:
                hyphae.get_nodes_within(t)[0]
            except nx.exception.NetworkXNoPath:
                to_remove_hyphae.add(hyphae)
                print(
                    f"error for hyphae {hyphae} on position {hyphae.end.pos(t),hyphae.root.pos(t)}"
                )
for hyphae in to_remove_hyphae:
    hyphaes.remove(hyphae)
exp_clean.hyphaes = hyphaes
for label in labels:
    exp_clean.nodes.append(Node(label, exp_clean))

removing  1456  nodes out of 16096 because they appear only in one timestep and are not within an identified hypha
[0.591831, 0.923236]
[0.705595, 1.041964]
[1.009608, 0.731445]
[1.7765, 1.458858]
[2.022328, 0.791296]
[4.224256, 0.883864]
[8.69064, 0.5647]
[14.173199, 0.436254]
[29.46564, 0.824076]
[43.285973]
[66.725277, 0.918873]
[93.80395]
[132.07266, 0.586922]
[188.26414, 0.487188]
[236.985742, 1.92942]
[359.584862, 0.835185]
[486.882388]
[626.062252, 0.80289]
[845.03342, 0.918932]
[1062.326034]
[1349.075964, 0.694512]
[1624.420581, 1.366316]
[1866.459474, 0.486438, 0.477554, 0.705888]
[2376.112924, 0.825045, 1.06455, 0.395752]
[3076.835385, 1.902133, 1.555987]
[3732.101802, 2.168316]


In [24]:
plt.close("all")
node_problem = choice(to_remove)
t = ts[node_problem][0]
exp.plot([t], [[node_problem]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
plot_raw_plus(exp, t, [node_problem])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
exp.hyphaes = hyphaes

In [6]:
equ_class, ambig, connection = resolve_ambiguity(exp_clean.hyphaes)
new_graph, newposs = relabel_nodes_after_amb(
    connection, exp_clean.nx_graph, exp_clean.positions
)

In [7]:
exp_clean.nx_graph = new_graph
exp_clean.positions = newposs
exp_clean.nodes = []

In [12]:
problems = []
for hyph in exp_clean.hyphaes:
    roots = []
    hyph = hyph
    tip = hyph.end
    for t in tip.ts():
        #             print(t,tip)
        if tip.degree(t) == 1:
            root, edges, nodes = hyph.get_edges(t, 200)
            roots.append(root)
    occurence_count = Counter(roots)
    if len(tip.ts()) > 0 and occurence_count.most_common(2)[0][0] != hyph.root:
        problems.append(hyph)

In [13]:
len(problems)

903

In [11]:
solve_two_ends = resolve_ambiguity_two_ends(exp_clean.hyphaes)

978 hyphae with two ends have been detected
Among the 6854, 966 hyphaes had two real ends, 690 ambiguity were solved by finding a degree 4 node


In [19]:
hyphae_two_ends = [hyph for hyph in exp.hyphaes if hyph.root.degree(hyph.ts[0]) == 1]

In [20]:
len(hyphae_two_ends)

978

In [34]:
hyphae = choice(hyphae_two_ends)
hyphae.root.degree(hyphae.ts[0])

1

In [35]:
plt.close("all")
t0 = hyphae.ts[0]
plot_raw_plus(exp, t0, [hyphae.end.label, hyphae.root.label])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
exp_clean.plot([t0], [[hyphae.end.label, hyphae.root.label]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
exp_clean = clean_exp_with_hyphaes(exp)

0.0
0.02884338044418806
0.05768676088837612
0.08653014133256418
0.11537352177675224
0.1442169022209403
0.17306028266512835
0.2019036631093164
0.23074704355350448
0.25959042399769255
0.2884338044418806
0.31727718488606865
0.3461205653302567
0.37496394577444475
0.4038073262186328
0.4326507066628209
0.46149408710700895
0.490337467551197
0.5191808479953851
0.5480242284395731
0.5768676088837612
0.6057109893279492
0.6345543697721373
0.6633977502163253
0.6922411306605134
0.7210845111047015
0.7499278915488895
0.7787712719930776
0.8076146524372656
0.8364580328814537
0.8653014133256418
0.8941447937698298
0.9229881742140179
0.9518315546582059
0.980674935102394
Detected problems during hyphae detection, 80 hyphaes have inconsistent root over time
removing  1456  nodes out of 16096 because they appear only in one timestep and are not within an identified hypha
[0.591831, 0.923236]
[0.705595, 1.041964]
[1.009608, 0.731445]
[1.7765, 1.458858]
[2.022328, 0.791296]
[4.224256, 0.883864]
[8.69064, 0.5647

In [28]:
len(hyphaes)

6854